In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
df = pd.read_csv("sg_companies_reviews.csv")

In [3]:
df.isna().sum()

Company Name         0
Overall Rating       0
Review Date          0
Review Title        66
Job Title            5
Job Details          0
Location          2763
Pros                 0
Cons                 0
dtype: int64

In [4]:
df

## dates 45215 is excel format, convert from 1jan1900


,Company Name,Overall Rating,Review Date,Review Title,Job Title,Job Details,Location,Pros,Cons
0,Amazon,5,12-Feb-24,"Good pay, hard work",Area Manager II,"Former Employee, more than 3 years","Romulus, MI","Pay, autonomy, plenty of benefits",It was hard work in a warehouse. Many days of ...
1,Amazon,5,10-Jan-16,You Get What You Put In,Anonymous Employee,Current Employee,"Seattle, WA","Really smart people, a lot of opportunity for ...",You have to be self motivated. NO ONE will hol...
2,Amazon,3,20-Feb-16,"Exciting Work, Abusive Culture",Senior Engineering Manager,"Current Employee, more than 8 years","Seattle, WA","Jeff Bezos and his ""S-Team"" are brilliant and ...","The management process is abusive, and I'm cur..."
3,Amazon,4,17-Dec-13,"Can be amazing for some people, horrible for o...",Software Development Manager,"Current Employee, more than 3 years","Seattle, WA",Amazon is doing lot's of cool stuff...but lots...,- You're responsible for your own career progr...
4,Amazon,5,23-Feb-18,An Amazing Place to Work,Software Development Manager,"Current Employee, less than 1 year","Phoenix, AZ","I've been at Amazon for a month now, and I've ...","No cons, so far - seriously. Like I said, I'm ..."
...,...,...,...,...,...,...,...,...,...
10030,NCR,4,45215,"Good, but could be better",Customer Engineer,"Former Employee, more than 1 year","Miami, FL","Management was good, solid training, opportuni...","Inconsistent schedule, growth within the compa..."
10031,NCR,5,45222,Great starting position,Sr. Software Support Engineer,Former Employee,"Atlanta, GA",Learn a lot Develop skills,There was a lot of overtime
10032,NCR,4,45143,"Great Company, on call rotation",ATM Technician,"Former Employee, more than 1 year","Springfield, MO","Great Company. Provided a vehicle, and covered...","Lots of travel, and rotating ""On call"". Very l..."
10033,NCR,1,45282,Na,Analyst,"Former Employee, more than 3 years","Atlanta, GA",Located in midtown and nice building,"No pay increase, revolving door of managers, a..."


In [43]:
def format_date(x, input_format = "%d-%b-%y", ignore_errors = False):
    try:
        return datetime.strptime(x, input_format)
    except Exception as e:
        if ignore_errors:
            return x
        else:
            raise e

def clean_df_dates(df, date_col, **args):
    df[date_col] = df[date_col].apply(lambda x: format_date(x, **args))
    return df

df = clean_df_dates(df, "Review Date", ignore_errors = True)

In [5]:
from cleantext import clean
import emoji

def convert_emoji_to_text(emoji_text):
    try:
        text_with_aliases = emoji.demojize(emoji_text, delimiters=("<", ">"))
    except:
        return emoji_text
    return text_with_aliases

def is_ascii(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
        return True
    except UnicodeDecodeError:
        return False

def get_non_ascii_strings(df, col):
    # Use apply function to check each value in the specified column
    non_ascii_mask = df[col].apply(lambda x: not is_ascii(str(x)))

    # Return a DataFrame containing non-ASCII strings
    return df[non_ascii_mask]

def clean_text(df, col):
    df = df.copy()
    df[col] = df[col].apply(lambda x: clean(x, no_emoji=False, no_punct=True))
    df[col] = df[col].apply(lambda x: convert_emoji_to_text(x))
    return df

text_columns = ['Review Title', 'Job Title', 'Job Details', 'Pros', 'Cons']
df_clean = df.copy()
for column_to_check in text_columns:
    df_clean = clean_text(df_clean, column_to_check)
    result = get_non_ascii_strings(df_clean, column_to_check)
    print(f"Non-ASCII strings in '{column_to_check}':")
    print(len(result))
    if len(result) > 0:
        display(result)


Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


Non-ASCII strings in 'Review Title':
0
Non-ASCII strings in 'Job Title':
0
Non-ASCII strings in 'Job Details':
0
Non-ASCII strings in 'Pros':
0
Non-ASCII strings in 'Cons':
0


In [59]:
print(df.iloc[4088]['Review Title'])
print("==========================")
print("After cleaning")
print("==========================")
print(df_clean.iloc[4088]['Review Title'])

💩Jaipur sitapura uper management
After cleaning
<pile_of_poo>jaipur sitapura uper management
